### *IPCC SR15 scenario assessment*

<img style="float: right; height: 100px; margin-top: 10px;" src="../_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="../_static/IAMC_logo.jpg">

# Analysis of carbon capture and sequestration (CCS)

This notebook computes indicators and diagnostics of the deployment of CCS by fuel
and the total amount of CO2 stored as shown in **Figure 2.17**
in the IPCC's _"Special Report on Global Warming of 1.5°C"_.

The scenario data used in this analysis can be accessed and downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).

## Load `pyam` package and other dependencies

In [ ]:
import pandas as pd
import numpy as np
import warnings
import io
import itertools
import yaml
import math
import matplotlib.pyplot as plt
%matplotlib inline
import pyam

from utils import boxplot_by_cat

## Import scenario data, categorization and specifications files

The metadata file must be generated from the notebook `sr15_2.0_categories_indicators` included in this repository.  
If the snapshot file has been updated, make sure that you rerun the categorization notebook.

The last cell of this section loads and assigns a number of auxiliary lists as defined in the categorization notebook.

In [ ]:
sr1p5 = pyam.IamDataFrame(data='../data/iamc15_scenario_data_world_r1.xlsx')

In [ ]:
sr1p5.load_metadata('sr15_metadata_indicators.xlsx')

In [ ]:
with open("sr15_specs.yaml", 'r') as stream:
    specs = yaml.load(stream)

rc = pyam.run_control()
for item in specs.pop('run_control').items():
    rc.update({item[0]: item[1]})
cats = specs.pop('cats')
all_cats = specs.pop('all_cats')
subcats = specs.pop('subcats')
all_subcats = specs.pop('all_subcats')
plotting_args = specs.pop('plotting_args')
marker= specs.pop('marker')

## Downselect scenario ensemble to categories of interest for this assessment

Use all years (as of 2020) to correctly compute the total amount of CO2 stored in each scenario.

In [ ]:
years = range(2020, 2101, 5)

In [ ]:
cats.remove('Above 2C')

In [ ]:
df = sr1p5.filter(category=cats, year=years)

## Set specifications for filter and plotting

In [ ]:
plot_ylabel = '{} with CCS (EJ)'
save_name = 'output/fig2.17{}.{}'
figure_format = 'png'

In [ ]:
filter_args = dict(df=sr1p5, category=cats, marker=None, join_meta=True)

In [ ]:
def plotting_args(name, panel_label=None, filetype=figure_format):
    return {'categories': cats, 'column': 'category', 'years': range(2020, 2101, 10), 'add_marker': marker,
            'ylabel': plot_ylabel.format(name),
            'save': save_name.format(name if panel_label is None else '{}_{}'.format(panel_label, name), filetype)}

## Add IEA's 'Faster Transition Scenario' to the set of marker scenarios for comparison

In [ ]:
m = 'IEA WEM'
col = 'marker'
sr1p5.set_meta(m, col,
               sr1p5.filter(model='IEA World Energy Model 2017',
                            scenario='Faster Transition Scenario'))
rc.update({'marker': {col: {m: 'o'}},
           'c': {col: {m: 'red'}},
           'edgecolors': {col: {m: 'black'}}}
         )
marker += [m]

## Extract CCS timeseries data by fuel

In [ ]:
ccs_bio = df.filter(variable='Primary Energy|Biomass|Modern|w/ CCS').timeseries()

In [ ]:
name = 'bioenergy'
fig = boxplot_by_cat(
    pyam.filter_by_meta(ccs_bio, **filter_args),
    **plotting_args(name, 'a'))

In [ ]:
ccs_coal = df.filter(variable='Primary Energy|Coal|w/ CCS').timeseries()

In [ ]:
name = 'coal'
boxplot_by_cat(
    pyam.filter_by_meta(ccs_coal, **filter_args),
    **plotting_args(name, 'b'), legend=False)

In [ ]:
ccs_gas = df.filter(variable='Primary Energy|Gas|w/ CCS').timeseries()
ccs_gas.index = ccs_gas.index.droplevel([2, 3, 4])

In [ ]:
name = 'gas'
boxplot_by_cat(
    pyam.filter_by_meta(ccs_gas, **filter_args),
    **plotting_args(name, 'c'), legend=False)

## Compute cumulative CO2 stored

In [ ]:
ccs = (
    df.filter(variable='Carbon Sequestration|CCS')
    .convert_unit({'Mt CO2/yr': ('Gt CO2/yr', 0.001)})
    .timeseries()
)

In [ ]:
cum_ccs = pd.DataFrame()
for i in range(2020, 2100, 10):
    cum_ccs[i] = ccs.apply(pyam.cumulative, raw=False, axis=1, first_year=2020, last_year=i)

In [ ]:
boxplot_by_cat(
    pyam.filter_by_meta(cum_ccs, **filter_args),
    categories=cats, column='category', years=range(2020, 2100, 10),
    ylabel='cumulative CO2 stored (Gt)',
    save=save_name.format('d_cumulative_ccs', figure_format),
    ymax=2050, add_marker=marker, legend=False)